In [3]:
#Loading Packages
import pandas as pd
import numpy as np
from pandas import json_normalize
import requests
import json
from datetime import datetime, timedelta

In [4]:
#Creating date variables for later use
yesterday = datetime.now() - timedelta(1)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')

thirtydays = datetime.now() - timedelta(36)
thirtydays = datetime.strftime(thirtydays, '%Y-%m-%d')

In [5]:
#Calling the API with a 400000 limit, may need to be expanded as the data set grows or may need to page if limit is exceeded
api_url = "https://healthdata.gov/resource/j8mb-icvb.json?$limit=400000"
r = requests.get(api_url)


In [6]:
#Taking the called json data, converting to a dataframe
json = r.json()
df = pd.DataFrame(json)
print(df)
#Converting datatypes to necessary forms
df['date'] = pd.to_datetime(df['date'])
df = df.astype({'new_results_reported':'int'})
df = df.astype({'total_results_reported':'int'})
#Question 1 Result
#df.loc[df['date'] == yesterday ,'total_results_reported'].sum()
df.loc[df['date'] == df['date'].max() ,'total_results_reported'].sum()
#So theres a few things here that were interesting, the total is a rolling number so you only need to call the latest one to get the desired result. 
#Additionally this API appears to update every monday potentially? So getting down to yesterday reliably does not seem like its often possible. 
#Because of this I included two commands, one to call yesterday if that data is in the dataframe, and if not the more applicable one which is just getting
#the sum of the total results reported on the latest day.

       state state_name state_fips fema_region overall_outcome  \
0         AL    Alabama         01    Region 4        Negative   
1         AL    Alabama         01    Region 4        Positive   
2         AL    Alabama         01    Region 4        Negative   
3         AL    Alabama         01    Region 4        Positive   
4         AL    Alabama         01    Region 4        Negative   
...      ...        ...        ...         ...             ...   
188434    WY    Wyoming         56    Region 8        Negative   
188435    WY    Wyoming         56    Region 8        Positive   
188436    WY    Wyoming         56    Region 8    Inconclusive   
188437    WY    Wyoming         56    Region 8        Negative   
188438    WY    Wyoming         56    Region 8        Positive   

                           date new_results_reported total_results_reported  
0       2020-03-01T00:00:00.000                   96                     96  
1       2020-03-01T00:00:00.000                   1

694336881

In [8]:
#Question Two process
#Filtering down to only positive results over last 30 days and grouping by date
df2_positive_trunc=df.loc[(df['overall_outcome'] == "Positive") & (df["date"] >= thirtydays)]
df2_positive_agg=df2_positive_trunc[['date','new_results_reported']].groupby(['date']).sum()
#Calculating the rolling mean and results
df2_positive_agg['rolling_mean'] = df2_positive_agg['new_results_reported'].rolling(window=7).mean()
display(df2_positive_agg.tail(30))
#A few things here as well, the offset 4 days of missing data does make this a little weird, I wasn't sure whether it was more correct to go 30 days back
#in the available data or 30 days from today and be missing a few values. I went with the latter I assume both approaches could be fine depending on 
#what people are looking for. This one is a little tricky because there are inconclusive results that I assume get updated later or retested and sent in as
#positive or negative for some states. So while this mean does get a snapshot of something close its worth being aware this may not be the true number of 
#positive tests since I don't know how each state handles unsure results.

C:\Users\cloon\AppData\Local\Temp\ipykernel_20332\3721394359.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  freq='1D', sort=True)).sum()


,new_results_reported,rolling_mean
date,,
2023-04-02,8693,NaN
2023-04-03,13879,NaN
2023-04-04,15471,NaN
2023-04-05,13658,12898.571429
2023-04-06,12103,12521.714286
2023-04-07,11124,12179.571429
2023-04-08,8791,11959.857143
2023-04-09,6981,11715.285714
2023-04-10,12279,11486.714286


In [151]:
#Question Three process
#Same idea of truncating down to the necessary range for two seperate data sets
df3_datetrunc=df.loc[(df["date"] >= thirtydays)]
df3_positive_datetrunc=df.loc[(df['overall_outcome'] == "Positive") & (df["date"] >= thirtydays)]
#Grouping both of those data sets to results reported
df3_dateagg=df3_datetrunc[['state_name','new_results_reported']].groupby(['state_name']).sum()
df3_positive_dateagg=df3_positive_datetrunc[['state_name','new_results_reported']].groupby(['state_name']).sum()
#Merging the data & calculating the percentage
merged_df3 = pd.merge(df3_dateagg, df3_positive_dateagg, how='outer', on = ['state_name'])
merged_df3_posrate = mergeddf3.assign(positivity_rate=lambda x: x.new_results_reported_y / x.new_results_reported_x * 100)
#Cleaning data and filtering to the necessary range
merged_df3_posrate = merged_df3_posrate.drop(columns=['new_results_reported_x','new_results_reported_y'])
merged_df3_clean=merged_df3_posrate.sort_values(by = 'positivity_rate', ascending = False).head(10)
display(merged_df3_clean)

#So this approach is a bit convoluted, I split the data sets and merged them back to get the two fields needed for calculating. I am sure thats not 
#the absolute ideal way compared to having some redundant values with the positives and trimming back from there. For data the size of the truncated 
#things we are working with I think the approach is fine but if it scaled to larger or more complex data a solution that does not require loading the 
#dataset twice and joining it on itself is obviously preferred. This was just what made sense in my head going step by step through the cleaning process. 
#As for insights it looks like Iowa only reports positive results meaning they seem to be much higher but would surely be more in line with the rest if 
#they reported everything. Additionally those inconclusive results might not be used by every state, and depending on where they would get bucketed by 
#those states that could swing their positivity rate somewhat.

,positivity_rate
state_name,
Iowa,100.000000
U.S. Virgin Islands,21.839080
Missouri,20.530434
South Dakota,18.234203
Oklahoma,12.918975
Puerto Rico,12.671575
Hawaii,11.772240
New Mexico,11.537414
Nevada,11.122692
